# Goals for AIGAME - RPS (Rock, Paper, Scissors) Part 6

In part 5
* We looked at a sample Iris dataset
* We explored methods for viewing/analyzing that data
* We applied a Decision Tree machine learning algorithm to determine the type of Iris based on measurements on the flower
* Finally we tried to apply a Decision Tree to predicting the outcome of a RPS game.

But we are not interestd in determining the outcome of a Rock Paper Scissors game. We already know how to do that and so we don't need AI help. Rather we want the AI to play the game for us, to choose whether the AI should choose Rock, Paper or Scissors.

### Does it make sense for an AI to play Rock, Paper, Scissors?
I know, I know. You might be saying that it makes no sense for an AI to play this game because, in theory, the game is totally random, like flipping a coin.

In theory it should be, but it's not in practice. For example, I tend to choose ROCK when I played the AI, I choose ROCK whenever I am unsure what to play. Also, if the AI beats me with the same choice twice in a row, I tend to assume it will continue to play that way and choose the move that will win that case.

Most people have patterns to their play and that is what the AI can learn. In my case, it should learn that I tend to prefer ROCK and hence it should prefer PAPER since PAPER beats/covers ROCK

We saw in the Iris dataset that measurements of the width and length of the sepal and petal of an Iris flow is useful in determining the type of Iris flower (target).

So what information would our AI want? Let's brainstorm:
* Percent of game history that the human chose ROCK
* Percent of game history that the human chose PAPER
* What did the human play last?
* What was the last outcome?

### Goal - we want to create a dataset which the AI can actually use



In [1]:
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split


# Build Experience - how to choose useful feature data

### Let's use the following data (features) for our RPS AI:
1.  Human response for the last 3 games (3 values)
1.  AI response for the last 3 games (3 values)
1.  Outcome for the last 3 games
1.  % Human chooses Rock in last 10 games
1.  % Human chooses Paper in last 10 games
1.  % Human chooses Scissors in last 10 games
1.  % Human chooses Rock in last 30 games
1.  % Human chooses Paper in last 30 games
1.  % Human chooses Scissors in last 30 games

### Let's create a dataframe which holds the above information

In [2]:
# Let's make a function which enters the above information into a DataFrame 
def addToRPSDataDS(ds, hL1,hL2,hL3,aiL1,aiL2,aiL3,outcome1,outcome2,outcome3,pctHumanRock10,pctHumanPaper10,pctHumanScissors10, 
                  pctHumanRock30, pctHumanPaper30, pctHumanScissors30, aiWinningMove):
    newval = {'HumanL1':[hL1], 'HumanL2':[hL2], 'HumanL3':[hL3], 'AIL1':[aiL1],'AIL2':[aiL2],'AIL3':[aiL3],'Outcome1':[outcome1], 'Outcome2':[outcome2],
             'Outcome3':[outcome3], 'PctHumanRock10': [pctHumanRock10], 'PctHumanPaper10': [pctHumanPaper10], 
              'PctHumanScissors10': [pctHumanScissors10], 'PctHumanRock30': [pctHumanRock30], 'PctHumanPaper30': [pctHumanPaper30], 
              'PctHumanScissors30': [pctHumanScissors30], 'AIWinningMove':[aiWinningMove]}
    retvalue = ds
    if (ds is None):
        retvalue = pd.DataFrame(newval)
    else:
        ds.loc[len(ds.index)] = [hL1,hL2,hL3,aiL1,aiL2,aiL3,outcome1,outcome2,outcome3,pctHumanRock10,pctHumanPaper10,
                                pctHumanScissors10,pctHumanRock30, pctHumanPaper30,pctHumanScissors30, aiWinningMove]
    return retvalue

In [3]:
# Let's test it out with some sample data
dsRPSData = addToRPSDataDS(None, 1,2,3,2,3,1,1,1,1,2.0,2.0,2.0,3.0,2.0,3.4,1)

dsHold = dsRPSData
dsRPSData = addToRPSDataDS(dsRPSData, 2,3,1,1,1,1,-1,1,0,2.0,2.0,2.0,3.0,2.0,3.4,1)
print( dsHold is dsRPSData)
print(dsRPSData)



True
   HumanL1  HumanL2  HumanL3  AIL1  AIL2  AIL3  Outcome1  Outcome2  Outcome3  \
0      1.0      2.0      3.0   2.0   3.0   1.0       1.0       1.0       1.0   
1      2.0      3.0      1.0   1.0   1.0   1.0      -1.0       1.0       0.0   

   PctHumanRock10  PctHumanPaper10  PctHumanScissors10  PctHumanRock30  \
0             2.0              2.0                 2.0             3.0   
1             2.0              2.0                 2.0             3.0   

   PctHumanPaper30  PctHumanScissors30  AIWinningMove  
0              2.0                 3.4            1.0  
1              2.0                 3.4            1.0  


### Now we want to use game history to create the above quantities.
We know how to determine the last three human moves, and the last three AI moves and the last three game outcomes. We can get that from our file which holds game history.

But how do we get the percent of the time the user selected ROCK over the last 10 games. This is a running  number which will change as each game is added. How do we get that?

### Remember when we talked about classes? And how we weren't going to use them? 

Fooled you, let's define a Python class for calculating running averages.

In [4]:
class RunningAverage:
    '''
    A class to determine a running average
    '''
    def __init__(self, windowSize, minPoints, defaultValue):
        '''
        Initializer for the class
        -windowSize is the size of our average windows. E.g. 10 = running average of 10 numbers
        -minPoints is the number of entries before we do the running calculation
        -defaultValue is the number we will return if we don't have enough points to match minPoints
        '''
        self.windowSize = windowSize
        self.minPoints = minPoints
        self.defaultValue = defaultValue
        self.values = [] #empty array
    def addToRunningAverage(self,val):
        '''
        add val to the running average
        '''
        self.values.insert(0,val)
        if len(self.values) > self.windowSize:
            self.values.pop()
    def getRunningAverage(self):
        '''
        return the current running average. 
            If no values, return None
            If not enough values, return defaultValue
            Else return the running average
        '''
        #print(len(self.values))
        if len(self.values) == 0:
            return None
        elif len(self.values) < self.minPoints:
            return self.defaultValue
        else:
            series1 = pd.Series(data=self.values)
            retvalue = series1.mean()
            return retvalue
            
    


In [5]:
# Let's test out the above function
ave10 = RunningAverage(windowSize=10, minPoints=3, defaultValue=0.33333)
print('We should get None, no points',ave10.getRunningAverage())
ave10.addToRunningAverage(1.0)
print('Should get default value',ave10.getRunningAverage())
ave10.addToRunningAverage(2.0)
print('Should get default value',ave10.getRunningAverage())
ave10.addToRunningAverage(1.0)
print('Now we have 3 points, we should get the average of 1,2 and 1=1.3333',ave10.getRunningAverage())
ave10.addToRunningAverage(2.0)
print('Now add 2 to the sum, should be 1,2,1 and 2 = 6/4 = 1.5', ave10.getRunningAverage())


We should get None, no points None
Should get default value 0.33333
Should get default value 0.33333
Now we have 3 points, we should get the average of 1,2 and 1=1.3333 1.3333333333333333
Now add 2 to the sum, should be 1,2,1 and 2 = 6/4 = 1.5 1.5


# Revisit: Feature vs Target
As we discussed before, the 'Target' is the value which the AI is trying to determine. The 'Feature Data' is the data which it can use to do so.
In the case of Iris flowers, the 'Feature Data' was the length and width of the sepal and petal of the flower. The 'Target' was the type of Iris.

For RPS, we discussed the feature data above. What is the target? Well, since the AI is playing the game, the 'Target' is the winning move, a selection of Rock, Paper or Scissors which will beat the human's choice (on average).

So let's create a function which determines the 'Target'

In [6]:
###################
# Rock, Paper or Scissors
ROCK=1
PAPER=2
SCISSORS=3
#AI Win, Human Win or DRAW
DRAW=0
AI_WIN=1
HUMAN_WIN=-1
#####################################
def DetermineAIWinningMove(humanResponse):
    if humanResponse == ROCK:
        return PAPER # PAPER beats ROCK
    elif humanResponse == PAPER:
        return SCISSORS #SCISSORS beats/cuts PAPER
    else:
        return ROCK #ROCK breaks scissors
def ConvertMoveCodeToString(rpsCode):
    if rpsCode == ROCK:
        return 'ROCK'
    elif rpsCode == PAPER:
        return 'PAPER'
    elif rpsCode == SCISSORS:
        return 'SCISSORS'
    else:
        return 'ILLEGAL CODE'

In [7]:
#Let's test it out
print('If human chooses ROCK, then AI should choose', ConvertMoveCodeToString(DetermineAIWinningMove(ROCK)))
print('If human chooses PAPER, then AI should choose', ConvertMoveCodeToString(DetermineAIWinningMove(PAPER)))
print('If human chooses SCISSORS, then AI should choose', ConvertMoveCodeToString(DetermineAIWinningMove(SCISSORS)))

If human chooses ROCK, then AI should choose PAPER
If human chooses PAPER, then AI should choose SCISSORS
If human chooses SCISSORS, then AI should choose ROCK


# Execute
We will now read the game history and turn it into something we can input to our AI: a DataFrame which represents game history statistics including running averages and recent plays.

In [8]:
RPS_RESULTS_PATH = 'C:/Users/LEN320/Google Drive/DEV/Python/rps_results.xls'
def LoadGameResults(fileLoadPath):
    '''This function takes one argument (the path of the saved results) and returns a DataFrame'''
    retvalue = pd.read_excel(fileLoadPath, sheet_name="RPS Results")
    return retvalue

dsResults = LoadGameResults(RPS_RESULTS_PATH)
dsRPSData = None 
pctHumanRock10 = RunningAverage(windowSize=10, minPoints=3, defaultValue=0.33333)
pctHumanPaper10 = RunningAverage(windowSize=10, minPoints=3, defaultValue=0.33333)
pctHumanScissors10 = RunningAverage(windowSize=10, minPoints=3, defaultValue=0.33333)
pctHumanRock30 = RunningAverage(windowSize=30, minPoints=3, defaultValue=0.33333)
pctHumanPaper30 = RunningAverage(windowSize=30, minPoints=3, defaultValue=0.33333)
pctHumanScissors30 = RunningAverage(windowSize=30, minPoints=3, defaultValue=0.33333)
HumanL2 = HumanL3 = AIL2 = AIL3 = 1 #default to rock
LastAIMove = LastHumanMove = HumanL1 = AIL1 = 1 # default all to ROCK
LastOutcome = Outcome2 = Outcome3 = Outcome1 =  0 #draw
for i in dsResults.index:
    #Data for this round, note that we will NOT store the Human or AI move for this round, only AIWinningMove (Target)
    ThisHumanMove = dsResults.at[i,'Human Response']
    ThisAIMove = dsResults.at[i,'AI Response'] 
    ThisOutcome = dsResults.at[i,'Outcome']
    AIWinner = DetermineAIWinningMove(ThisHumanMove)
    
    
    #Update history for last 3 moves
    HumanL3 = HumanL2
    HumanL2 = HumanL1
    HumanL1 = LastHumanMove
    AIL3 = AIL2
    AIL2 = AIL1
    AIL1 = LastAIMove   
    Outcome3 = Outcome2
    Outcome2 = Outcome1   
    Outcome1 = LastOutcome
    
    #Update running averages for the last round
    pctHumanRock10.addToRunningAverage(1.0 if HumanL1==ROCK else 0.0)
    pctHumanPaper10.addToRunningAverage(1.0 if HumanL1==PAPER else 0.0)
    pctHumanScissors10.addToRunningAverage(1.0 if HumanL1==SCISSORS else 0.0)

    pctHumanRock30.addToRunningAverage(1.0 if HumanL1==ROCK else 0.0)
    pctHumanPaper30.addToRunningAverage(1.0 if HumanL1==PAPER else 0.0)
    pctHumanScissors30.addToRunningAverage(1.0 if HumanL1==SCISSORS else 0.0)  
    
    #Update last move
    LastAIMove = ThisAIMove
    LastHumanMove = ThisHumanMove
    LastOutcome = ThisOutcome
    
    #Fill the data frame, note that we record the AI Winning Move because that is the target. But the rest is historical. 
    #We want the AI to choose based on history only
  
    
    #print('winning move on', HumanL1,'is',AIWinner, 'Rock10Human=',pctHumanRock10.getRunningAverage())
    dsRPSData = addToRPSDataDS(dsRPSData, HumanL1, HumanL2, HumanL3, AIL1, AIL2, AIL3, Outcome1, Outcome2, Outcome3,
                               pctHumanRock10.getRunningAverage(), pctHumanPaper10.getRunningAverage(), pctHumanScissors10.getRunningAverage(),
                               pctHumanRock30.getRunningAverage(), pctHumanPaper30.getRunningAverage(), pctHumanScissors30.getRunningAverage(),
                              AIWinner)

### Let's save these results in case we want to look at them in more detail

In [9]:
def SaveGameResults(fileSavePath, resultsDataFrame):
    '''This function takes two arguments:
    1) fileSavePath - the full path to the file which will hold the results, e.g. C:\RPS_AI\Results.csv (it will be a csv)
    2) resultsDataFrame - a DataFrame which holds all the results
    '''
    with pd.ExcelWriter(fileSavePath) as writer:
        resultsDataFrame.to_excel(writer, sheet_name="RPS Results", index=False)
        writer.save()
dsRPSData.describe()
GAME_RESULTS_PATH = 'C:/Users/LEN320/Google Drive/DEV/Python/rps_game.xls'
SaveGameResults(GAME_RESULTS_PATH, dsRPSData)

### We should take a look at our data for any issues

In [10]:
dsRPSData.describe()

,HumanL1,HumanL2,HumanL3,AIL1,AIL2,AIL3,Outcome1,Outcome2,Outcome3,PctHumanRock10,PctHumanPaper10,PctHumanScissors10,PctHumanRock30,PctHumanPaper30,PctHumanScissors30,AIWinningMove
count,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000
mean,1.575758,1.515152,1.515152,2.000000,1.969697,1.939394,0.151515,0.181818,0.151515,0.616294,0.230171,0.153535,0.683560,0.212962,0.103478,2.000000
std,0.791766,0.755034,0.755034,0.829156,0.847233,0.863836,0.755034,0.726918,0.712444,0.180283,0.071523,0.141896,0.131469,0.054162,0.094049,0.661438
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-1.000000,-1.000000,-1.000000,0.333330,0.100000,0.000000,0.333330,0.100000,0.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.500000,0.200000,0.000000,0.619048,0.190476,0.000000,2.000000
50%,1.000000,1.000000,1.000000,2.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.600000,0.200000,0.100000,0.666667,0.214286,0.111111,2.000000
75%,2.000000,2.000000,2.000000,3.000000,3.000000,3.000000,1.000000,1.000000,1.000000,0.800000,0.300000,0.300000,0.769231,0.233333,0.166667,2.000000
max,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,1.000000,1.000000,1.000000,0.900000,0.333333,0.400000,0.900000,0.333333,0.333330,3.000000


1. Look at the max/min of each column. It makes sense that the min for the player moves, ai moves and winning move to be 1 and max to be 3.
1. Also make sense for the outcome to range from -1 to 1.
1. The counts are all 33 so we assume that our history included 33 games and it's good that all columns have 33 values
1. Note that the average of the outcomes is positive which implies that the AI won on average :( even when we used a random number generator


We have created a DataFrame that holds both our 'Target' (Winning move for AI) and all of the 'Feature Data' which we thought might be useful in determining the Target. Let's explorer the data a bit.

Let's look at the last 5 records


In [11]:
dsRPSData.tail(5)

,HumanL1,HumanL2,HumanL3,AIL1,AIL2,AIL3,Outcome1,Outcome2,Outcome3,PctHumanRock10,PctHumanPaper10,PctHumanScissors10,PctHumanRock30,PctHumanPaper30,PctHumanScissors30,AIWinningMove
28,1.0,1.0,1.0,1.0,1.0,2.0,0.0,0.0,1.0,0.6,0.2,0.2,0.655172,0.206897,0.137931,3.0
29,2.0,1.0,1.0,3.0,1.0,1.0,1.0,0.0,0.0,0.6,0.3,0.1,0.633333,0.233333,0.133333,1.0
30,3.0,2.0,1.0,3.0,3.0,1.0,0.0,1.0,0.0,0.6,0.3,0.1,0.600000,0.233333,0.166667,2.0
31,1.0,3.0,2.0,2.0,3.0,3.0,1.0,0.0,1.0,0.6,0.3,0.1,0.633333,0.200000,0.166667,1.0
32,3.0,1.0,3.0,2.0,2.0,3.0,-1.0,1.0,0.0,0.6,0.2,0.2,0.600000,0.200000,0.200000,1.0


You can see that in last 5 games, I was clearly preferring ROCK as the PctHumanRock10 was >= 0.6 and even PctHumanRock30 >=0.6


We can also look at correlations between the numbers. A machine learning algorithm will take advantage of correlations between the feature data and the target ('the winning move' in our case).

In [12]:
# We show a heat map (or correlation map) for the data in the DataFrame we just created.
# You can see what values are correlated to each other. Of special interest is data that is correlated
# with our target (AIWinningMove). If we needed to reduce the amount of input data, we could use this map
# to guide our choices. But for now we leave it all.
import seaborn as sns
correlation = dsRPSData.corr()
#plt.style.use("ggplot")
sns.heatmap(correlation, annot=True, cbar=True, cmap="RdYlGn")

Heatmap's are an interesting way to view correlations between data fields, but it is too cluttered for us to see. Let's only look at correlations with our target:
    

In [13]:
correlation.AIWinningMove

HumanL1              -0.298355
HumanL2               0.000000
HumanL3              -0.062574
AIL1                 -0.170941
AIL2                 -0.278823
AIL3                  0.000000
Outcome1              0.125148
Outcome2             -0.129989
Outcome3              0.066315
PctHumanRock10        0.034941
PctHumanPaper10      -0.374324
PctHumanScissors10    0.144281
PctHumanRock30        0.008710
PctHumanPaper30      -0.090563
PctHumanScissors30    0.039973
AIWinningMove         1.000000
Name: AIWinningMove, dtype: float64

None of the columns are strongly correlated with the AIWinningMove. But note that HumanL1 is anti-correlated and so is PctHumanPaper10.  I'm not sure what to make of that. So let's move on with the AI


In [14]:
#import matplotlib.pyplot as plt
#plt.xlabel('Human Previous Move (HumanL1)')
#plt.ylabel('AI Winning Move This Turn')
#plt.scatter(dsRPSData['HumanL1'],dsRPSData['AIWinningMove'], color='red')
#plt.show()

In [15]:
Target = dsRPSData['AIWinningMove'] #This is what the AI needs to determine: the winning move
Features = dsRPSData.drop(['AIWinningMove'], axis=1) #This is the data which can be used to learn the winning move.
#Note we remove AIWinningMove column as that is the answer and Features should not include the answer
Features.describe()

,HumanL1,HumanL2,HumanL3,AIL1,AIL2,AIL3,Outcome1,Outcome2,Outcome3,PctHumanRock10,PctHumanPaper10,PctHumanScissors10,PctHumanRock30,PctHumanPaper30,PctHumanScissors30
count,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000
mean,1.575758,1.515152,1.515152,2.000000,1.969697,1.939394,0.151515,0.181818,0.151515,0.616294,0.230171,0.153535,0.683560,0.212962,0.103478
std,0.791766,0.755034,0.755034,0.829156,0.847233,0.863836,0.755034,0.726918,0.712444,0.180283,0.071523,0.141896,0.131469,0.054162,0.094049
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-1.000000,-1.000000,-1.000000,0.333330,0.100000,0.000000,0.333330,0.100000,0.000000
25%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.500000,0.200000,0.000000,0.619048,0.190476,0.000000
50%,1.000000,1.000000,1.000000,2.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.600000,0.200000,0.100000,0.666667,0.214286,0.111111
75%,2.000000,2.000000,2.000000,3.000000,3.000000,3.000000,1.000000,1.000000,1.000000,0.800000,0.300000,0.300000,0.769231,0.233333,0.166667
max,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,1.000000,1.000000,1.000000,0.900000,0.333333,0.400000,0.900000,0.333333,0.333330


In [16]:
#As before, split the data in a training set and a test set. We will use 80% of the data for training.
X_train, X_test, y_train, y_test = train_test_split(Features, Target, test_size=0.20, shuffle=False)
X_train.shape

(26, 15)

In [23]:
#Create the DecisionTree and train it on the training data
clf = DecisionTreeClassifier(random_state=0 )
results = clf.fit(X_train,y_train)


In [24]:
#Let's determine the score (the fraction of time the AI chose the correct move)
results.score(X_test,y_test)

0.14285714285714285

###  When I ran this, I got a score of 0.14 or 14%. This is not good because random guessing should get a score of 1/3 or 33%. Let's see what would happen if we applied the score to the training set.


In [25]:
results.score(X_train,y_train)

1.0

### Hmm, so the algorithm got a 100% on the training data but only 29% on the test data. I have made a common machine learning mistake. But let's talk about that next time. For now, let's see what the Decision Tree is doing.

Remember we said that a Decision Tree has a set of rules, much like the game of 20 questions? So we can ask it to tell us those rules. This sets it apart from some other machine learning algorithms where it is much harder to tell what the algorithm is doing.

In [26]:
from sklearn.tree import _tree

def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    print("def tree({}):".format(", ".join(feature_names)))

    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print( "{}if {} <= {}:".format(indent, name, threshold))
            recurse(tree_.children_left[node], depth + 1)
            print("{}else:  # if {} > {}".format(indent, name, threshold))
            recurse(tree_.children_right[node], depth + 1)
        else:
            print("{}return {}".format(indent, tree_.value[node]))

    recurse(0, 1)

In [21]:
tree_to_code(results, feature_names=Features.columns)

def tree(HumanL1, HumanL2, HumanL3, AIL1, AIL2, AIL3, Outcome1, Outcome2, Outcome3, PctHumanRock10, PctHumanPaper10, PctHumanScissors10, PctHumanRock30, PctHumanPaper30, PctHumanScissors30):
  if PctHumanRock10 <= 0.894444465637207:
    if PctHumanScissors30 <= 0.17028985917568207:
      if HumanL2 <= 2.5:
        if Outcome2 <= 0.5:
          return [[ 0. 11.  0.]]
        else:  # if Outcome2 > 0.5
          if Outcome3 <= 0.5:
            if PctHumanPaper30 <= 0.19090908765792847:
              return [[0. 1. 0.]]
            else:  # if PctHumanPaper30 > 0.19090908765792847
              return [[3. 0. 0.]]
          else:  # if Outcome3 > 0.5
            return [[0. 2. 0.]]
      else:  # if HumanL2 > 2.5
        return [[1. 0. 1.]]
    else:  # if PctHumanScissors30 > 0.17028985917568207
      if Outcome2 <= 0.5:
        return [[0. 2. 1.]]
      else:  # if Outcome2 > 0.5
        return [[0. 0. 2.]]
  else:  # if PctHumanRock10 > 0.894444465637207
    return [[0. 0. 2.]]


From the above, we see that our training set had only 26 points. The code below has eight different return conditions. If we had had 8 points then one could easily write an algorithm specific to those points (e.g. if data=point1 return answer1). Let's look at the first condition:<br>
  if PctHumanRock10 <= 0.894444465637207:<br>
    if PctHumanScissors30 <= 0.17028985917568207:<br>
      if HumanL2 <= 2.5:<br>
        if Outcome2 <= 0.5:<br>
          return  0. 11.  0.<br>
Let's look at the corresponding data

In [28]:
training = X_train.copy()
training['answer'] = y_train
training.head()


,HumanL1,HumanL2,HumanL3,AIL1,AIL2,AIL3,Outcome1,Outcome2,Outcome3,PctHumanRock10,PctHumanPaper10,PctHumanScissors10,PctHumanRock30,PctHumanPaper30,PctHumanScissors30,answer
0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.333330,0.333330,0.33333,0.333330,0.333330,0.33333,3.0
1,2.0,1.0,1.0,1.0,1.0,1.0,-1.0,0.0,0.0,0.333330,0.333330,0.33333,0.333330,0.333330,0.33333,2.0
2,1.0,2.0,1.0,1.0,1.0,1.0,0.0,-1.0,0.0,0.666667,0.333333,0.00000,0.666667,0.333333,0.00000,2.0
3,1.0,1.0,2.0,3.0,1.0,1.0,-1.0,0.0,-1.0,0.750000,0.250000,0.00000,0.750000,0.250000,0.00000,2.0
4,1.0,1.0,1.0,3.0,3.0,1.0,-1.0,-1.0,0.0,0.800000,0.200000,0.00000,0.800000,0.200000,0.00000,2.0


In [36]:
cond1 = training['PctHumanRock10'] <= 0.894444465637207
cond2 = training['PctHumanScissors30'] <= 0.17028985917568207
cond3 = training['HumanL2'] <= 2.5
cond4 = training['Outcome2'] <= 0.5
allconds = cond1 & cond2 & cond3 & cond4
print(training[allconds])

    HumanL1  HumanL2  HumanL3  AIL1  AIL2  AIL3  Outcome1  Outcome2  Outcome3  \
2       1.0      2.0      1.0   1.0   1.0   1.0       0.0      -1.0       0.0   
3       1.0      1.0      2.0   3.0   1.0   1.0      -1.0       0.0      -1.0   
4       1.0      1.0      1.0   3.0   3.0   1.0      -1.0      -1.0       0.0   
5       1.0      1.0      1.0   1.0   3.0   3.0       0.0      -1.0      -1.0   
6       1.0      1.0      1.0   1.0   1.0   3.0       0.0       0.0      -1.0   
7       1.0      1.0      1.0   3.0   1.0   1.0      -1.0       0.0       0.0   
8       1.0      1.0      1.0   1.0   3.0   1.0       0.0      -1.0       0.0   
15      2.0      1.0      3.0   3.0   1.0   3.0       1.0       0.0       0.0   
17      3.0      1.0      2.0   3.0   1.0   3.0       0.0       0.0       1.0   
24      1.0      2.0      2.0   3.0   2.0   3.0      -1.0       0.0       1.0   
25      1.0      1.0      2.0   2.0   3.0   2.0       1.0      -1.0       0.0   

    PctHumanRock10  PctHuma

We see that all of the rows that satisfy these conditions all have one important thing in common: the correct response for all of them is 2 = Paper.<br>
So the algorithm did what it should, it found conditions that determine when Paper is a winning move. We now see a little of how the Decision Tree works; each leaf ends with a count of satisfying targets. In this case they are all Paper so there are no Rocks and no Scissors and hence 0,11,0.<br>
And so we also see the issue. With only 26 games of our training set, the algorithm is able to construct enough conditions to match each of them. Oh, we could add more games to the training set, but then wouldn't the algorithm make a more detailed tree? It would. The algorithm is over-fitting the data. So we need to set some limits on how detailed a tree can be constructed. We'll talke about that next time.